<a href="https://colab.research.google.com/github/blusewill/ytvideo-whisper/blob/dev/ytvideo_whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ytvideo-whisper

This Project uses the Original [Whisper](https://github.com/openai/whisper) Project instead of Decipher Project.

For Everyone who want to use the Original Whisper Project

# Usage

1. Check the Runtime Type is on GPU Mode in ``Runtime -> Change Runtime Type ``
1. Change the Settings at Settings Code Block
1. Click ``Runtime -> Run all`` (CTRL+F9)
1. Click on the Connect to Google Drive
1. And Wait for a moment your Generated Srt should be on ``Google Drive -> Whisper -> result``

## Add Google Drive Access 
(at Google Drive/Whisper)

In [1]:
from google.colab import drive
import os

path = '/content/gdrive/MyDrive/Whisper/'
path2 = '/content/gdrive/MyDrive/Whisper/result'
drive.mount('/content/gdrive')

if not os.path.exists(path):
  os.mkdir(path)
  os.mkdir(path2)


Mounted at /content/gdrive


# Install Components

Package : yt-dlp ffmpeg openai-whisper

In [2]:
! pip install openai-whisper yt-dlp ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.9/792.9 kB 18.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 88.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 95.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 112.5 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=796910 sha256=aa3f96357b98ead382937feffc492661267c2179993d0253a265a58f039c

# GPU Checkup

If Nothing shows up means you didn't enable GPU in the ``Runtime -> Change Runtime type``

In [3]:
! nvidia-smi

Wed May 24 06:50:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Settings


In [4]:
#@markdown # **Video Link Here**
# YouTube Link
YouTube_Video_Link = "https://www.youtube.com/watch?v=kfQC-sQxMcw" #@param {type:"string"}
# File Name Change
new_filename = "6-Month Health Transformation" #@param {type:"string"}
#@markdown Note: Leave it Blank to let it audio Detect the Language
# Change it to srt file type
new_filename = os.path.splitext(new_filename)[0] + ".srt"
new_filename2 = os.path.splitext(new_filename)[0] + ".txt"
# Change the Model of the Whisper
#@markdown # **Other Options**
shutdown_after_complete = "yes" #@param ['yes','no']
model_user = "medium" #@param ['tiny.en','tiny','base.en','base','small.en','small','medium.en','medium','large-v1','large-v2','large']

# Download the Video
using the yt-dlp and ffmpeg

In [5]:
import yt_dlp

ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
        'outtmpl': 'content/audio',
}
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([YouTube_Video_Link])

[youtube] Extracting URL: https://www.youtube.com/watch?v=kfQC-sQxMcw
[youtube] kfQC-sQxMcw: Downloading webpage
[youtube] kfQC-sQxMcw: Downloading android player API JSON
[youtube] kfQC-sQxMcw: Downloading player 41b8bed0
[info] kfQC-sQxMcw: Downloading 1 format(s): 251
[dashsegments] Total fragments: 1
[download] Destination: content/audio
[download] 100% of    9.39MiB in 00:00:00 at 26.93MiB/s              
[ExtractAudio] Destination: content/audio.mp3
Deleting original file content/audio (pass -k to keep)


# Generating the SRT file

In [6]:
import whisper

model = whisper.load_model(model_user)

print("Whisper model loaded.")


from whisper.utils import get_writer

filename = "audio.mp3"
input_directory = "content"
input_file = f"{input_directory}/{filename}"

result = model.transcribe(input_file, verbose=True)
transcription_root = "/content/gdrive/MyDrive/Whisper/result"

# Save as an SRT file
srt_writer = get_writer("srt", transcription_root,)
srt_writer(result, new_filename)

# Also Save as TXT file
srt_writer = get_writer("txt", transcription_root,)
srt_writer(result, new_filename2)

print("Generated SRT and txt file in the Whisper/result Folder")

100%|██████████████████████████████████████| 1.42G/1.42G [00:12<00:00, 120MiB/s]


Whisper model loaded.
Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: English
[00:00.000 --> 00:06.920]  It's been a little bit on the fitness update a lot of people have been following me for any period of time
[00:06.920 --> 00:12.360]  You know that I've been working out this past six months. What you may not have known is
[00:13.000 --> 00:17.720]  This I actually got diagnosed this past year with
[00:18.200 --> 00:25.520]  Gilbert's disease which means I can't drink alcohol and then also fatty liver and just really bad lipid panels and just overall
[00:25.840 --> 00:32.020]  Low low exercise just health in general my HDLs were also low which leads to you know
[00:32.020 --> 00:35.360]  Hey, I was sitting around like most computer guys out there
[00:35.360 --> 00:38.020]  I was sitting at a computer eight hours plus day
[00:38.020 --> 00:44.700]  And then I get off of my job or whatever I was doing and then I play gam

# Auto Shutdown the Runtime

In [7]:
if shutdown_after_complete.lower() == "yes":
    print("Shutting Down the Runtime Because Shutdown After Complete is on")
    from google.colab import runtime
    runtime.unassign()

Shutting Down the Runtime Because Shutdown After Complete is on
